In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import imgaug.augmenters as iaa




2023-07-27 16:26:01.862467: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 16:26:01.896560: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-27 16:26:01.897206: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-27 16:26:02.579574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [3]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)


In [4]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [5]:
augmenter = iaa.Sequential([
    iaa.GaussianBlur(sigma=(0, 1.0))
])

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
   preprocessing_function=augmenter.augment_image
)

train_dataset = datagen.flow(x_train[..., np.newaxis], y_train, batch_size=64)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val[..., np.newaxis], y_val)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test[..., np.newaxis], y_test)).batch(64)




In [6]:
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28, 1)),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [7]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_dataset, epochs=5, verbose=1, validation_data=val_dataset)


Epoch 1/5
844/844 [==============================] - 25s 30ms/step - loss: 0.7932 - accuracy: 0.7423 - val_loss: 0.3825 - val_accuracy: 0.8778
Epoch 2/5
844/844 [==============================] - 25s 29ms/step - loss: 0.4331 - accuracy: 0.8611 - val_loss: 0.2675 - val_accuracy: 0.9168
Epoch 3/5
844/844 [==============================] - 25s 30ms/step - loss: 0.3676 - accuracy: 0.8831 - val_loss: 0.2226 - val_accuracy: 0.9303
Epoch 4/5
844/844 [==============================] - 25s 30ms/step - loss: 0.3311 - accuracy: 0.8966 - val_loss: 0.2074 - val_accuracy: 0.9362
Epoch 5/5
844/844 [==============================] - 25s 30ms/step - loss: 0.3087 - accuracy: 0.9021 - val_loss: 0.2007 - val_accuracy: 0.9378


In [8]:
test_scores = model.evaluate(x_test, y_test, verbose=0)
test_accuracy, test_loss = test_scores[1], test_scores[0]

y_pred_probs = model.predict(x_test)
y_pred = tf.argmax(y_pred_probs, axis=1)
#y_pred = model.predict(x_test)

#precision = precision_score(y_test, y_pred, average=None)
#recall = recall_score(y_test, y_pred, average=None)
#f1 = f1_score(y_test, y_pred, average=None)
precision = precision_score(tf.argmax(y_test, axis=1), y_pred, average='weighted')
recall = recall_score(tf.argmax(y_test, axis=1), y_pred, average='weighted')
f1 = f1_score(tf.argmax(y_test, axis=1), y_pred, average='weighted')

print(f"test accuracy: {test_accuracy:.4f}")
print(f"test precision: {precision:.4f}")
print(f"test recall: {recall:.4f}")
print(f"test f1 Score: {f1:.4f}")

313/313 [==============================] - 0s 599us/step
test accuracy: 0.9392
test precision: 0.9405
test recall: 0.9392
test f1 Score: 0.9394


In [13]:
val_scores = model.evaluate(x_val, y_val, verbose=0)
val_accuracy, val_loss = val_scores[1], val_scores[0]
y_pred_probs = model.predict(x_val)
y_pred = tf.argmax(y_pred_probs, axis=1)

precision = precision_score(tf.argmax(y_val, axis=1), y_pred, average='weighted')
recall = recall_score(tf.argmax(y_val, axis=1), y_pred, average='weighted')
f1 = f1_score(tf.argmax(y_val, axis=1), y_pred, average='weighted')

print(f"val Accuracy: {val_accuracy:.4f}")
print(f"val Precision: {precision:.4f}")
print(f"val Recall: {recall:.4f}")
print(f"val F1 Score: {f1:.4f}")

188/188 [==============================] - 0s 635us/step
val Accuracy: 0.9378
val Precision: 0.9397
val Recall: 0.9378
val F1 Score: 0.9382
